# Model Evaluation and First Interpretability Pass

This notebook evaluates whether the modeling pipeline captures meaningful biological signal in drug response prediction.
The focus is on honest performance assessment, sanity checks, and a controlled first-pass interpretability analysis.

At this stage, the goal is **not** to optimize models or claim biological mechanisms,
but to determine whether the current feature space explains variability in drug response
beyond trivial baselines.


## Imports, paths, and minimal I/O checks

In [ ]:
# Import necessary libraries

from pathlib import Path
from typing import Dict, Tuple

import pandas as pd
import numpy as np

from sklearn.linear_model import Ridge, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import spearmanr


In [2]:
# Project paths (assumes this notebook lives in /notebooks)
PROJECT_ROOT = Path.cwd().resolve().parents[0]
DATA_PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"

In [3]:
# Input parquet files
EXPR_PATH = DATA_PROCESSED_DIR / "depmap_expression_matched.parquet"
PRISM_PATH = DATA_PROCESSED_DIR / "prism_auc_filtered.parquet"
DRUG_INDEX_PATH = DATA_PROCESSED_DIR / "drug_index.parquet"
CELL_META_PATH = DATA_PROCESSED_DIR / "cell_line_metadata.parquet"
SELECTED_DRUGS_PATH = DATA_PROCESSED_DIR / "selected_drugs.parquet"

paths = {
    "expression": EXPR_PATH,
    "prism": PRISM_PATH,
    "drug_index": DRUG_INDEX_PATH,
    "cell_meta": CELL_META_PATH,
    "selected_drugs": SELECTED_DRUGS_PATH,
}

In [4]:
# Check that all required parquet files exist
missing_files = [name for name, p in paths.items() if not p.exists()]
if missing_files:
    raise FileNotFoundError(
        "Missing required parquet files:\n"
        + "\n".join(f"- {name}" for name in missing_files)
    )

In [5]:
# Load parquets
expr_df = pd.read_parquet(EXPR_PATH)
prism_df = pd.read_parquet(PRISM_PATH)
drug_index_df = pd.read_parquet(DRUG_INDEX_PATH)
cell_meta_df = pd.read_parquet(CELL_META_PATH)
selected_drugs_df = pd.read_parquet(SELECTED_DRUGS_PATH)

In [6]:
# Minimal sanity prints
print("Expression shape:", expr_df.shape)
print("PRISM shape:", prism_df.shape)
print("Drug index shape:", drug_index_df.shape)
print("Cell metadata shape:", cell_meta_df.shape)
print("Selected drugs shape:", selected_drugs_df.shape)

display(selected_drugs_df.head())

print("\n✅ all parquet inputs loaded successfully.")

Expression shape: (751, 19220)
PRISM shape: (732066, 5)
Drug index shape: (1528, 7)
Cell metadata shape: (751, 2)
Selected drugs shape: (100, 7)


,broad_id,name,n_cell_lines,auc_mean,auc_std,auc_min,auc_max
0,BRD-K95142244-001-01-5,talazoparib,711,0.684250,0.195109,0.117250,1.807046
1,BRD-K50168500-001-07-9,canertinib,707,0.915852,0.153452,0.305531,1.792206
2,BRD-K33610132-001-02-9,rociletinib,704,0.959678,0.178074,0.535787,2.882057
3,BRD-A70858459-001-01-7,estramustine,699,1.004251,0.181686,0.492309,1.954327
4,BRD-K77625799-001-07-7,vandetanib,690,1.006869,0.196001,0.625323,2.260526



✅ all parquet inputs loaded successfully.


## Dataset Assembly for Evaluation

In this section, we reconstruct the modeling dataset in memory by combining
DepMap expression features with PRISM drug response measurements.

The dataset is restricted to the predefined MVP drug subset and to cell lines
with matched molecular and pharmacological data.


## Filter PRISM to MVP drugs + coverage checks

In [ ]:
# 1) Define MVP drug list
mvp_broad_ids = set(selected_drugs_df["broad_id"].dropna().unique())
print("MVP drugs:", len(mvp_broad_ids))

MVP drugs: 100
PRISM (MVP) shape: (68160, 5)
Unique PRISM MVP cell lines: 727
Unique PRISM MVP drugs: 100

Cell line overlap:
 - expression cells: 727
 - prism mvp cells: 727
 - shared: 727
 - shared vs prism (%): 100.0

PRISM (MVP + shared cell lines) shape: (68160, 5)


,broad_id,depmap_id,auc,name,join_id
1238,BRD-K95142244-001-01-5,ACH-001001,0.674423,talazoparib,ACH-001001
1239,BRD-K95142244-001-01-5,ACH-000956,0.740586,talazoparib,ACH-000956
1240,BRD-K95142244-001-01-5,ACH-000948,0.625397,talazoparib,ACH-000948



✅ Cell 02 passed: PRISM filtered to MVP drugs and matched cell lines.


In [ ]:
# 2) Filter PRISM to MVP drugs
prism_mvp = prism_df[prism_df["broad_id"].isin(mvp_broad_ids)].copy()
print("PRISM (MVP) shape:", prism_mvp.shape)

In [ ]:
# 3) Basic column checks (defensive)
required_prism_cols = ["depmap_id", "broad_id", "auc"]
missing = [c for c in required_prism_cols if c not in prism_mvp.columns]
if missing:
    raise ValueError(
        f"Missing required columns in prism_mvp: {missing}\n"
        f"Available columns: {list(prism_mvp.columns)}"
    )

In [ ]:
# 4) Coverage: how many unique cell lines and drugs are actually present?
n_prism_cells = prism_mvp["depmap_id"].nunique()
n_prism_drugs = prism_mvp["broad_id"].nunique()
print("Unique PRISM MVP cell lines:", n_prism_cells)
print("Unique PRISM MVP drugs:", n_prism_drugs)

In [ ]:
# 5) Intersect PRISM cell lines with expression cell lines
expr_cells = set(expr_df["depmap_id"].dropna().unique()) if "depmap_id" in expr_df.columns else set(expr_df.index)
prism_cells = set(prism_mvp["depmap_id"].dropna().unique())
shared_cells = prism_cells.intersection(expr_cells)

print("\nCell line overlap:")
print(" - expression cells:", len(expr_cells))
print(" - prism mvp cells:", len(prism_cells))
print(" - shared:", len(shared_cells))
print(" - shared vs prism (%):", round(100 * len(shared_cells) / max(1, len(prism_cells)), 2))

In [ ]:
# 6) Filter PRISM further to shared cell lines
prism_mvp = prism_mvp[prism_mvp["depmap_id"].isin(shared_cells)].copy()
print("\nPRISM (MVP + shared cell lines) shape:", prism_mvp.shape)

In [ ]:
# 7) Quick sanity checks
if prism_mvp["auc"].isna().any():
    n_na = int(prism_mvp["auc"].isna().sum())
    raise ValueError(f"'auc' contains NaNs after filtering: {n_na} rows.")

display(prism_mvp.head(3))

print("\n✅ Cell 02 passed: PRISM filtered to MVP drugs and matched cell lines.")

## Train / Test Split Strategy

To ensure a transparent and reproducible evaluation, models are assessed using
a fixed train/test split applied independently for each drug.

This strategy prioritizes signal detection and interpretability over variance
reduction, which will be addressed in later stages via cross-validation.

## Define evaluation split and MVP drug subset

In [ ]:
# Reproducibility
RANDOM_STATE = 42
TEST_SIZE = 0.2

Evaluation drugs (n=5):
 - BRD-A06352508-001-03-7
 - BRD-A06627858-236-03-0
 - BRD-A08545410-003-07-8
 - BRD-A22707317-001-10-8
 - BRD-A49035384-003-28-9

Drug BRD-A06352508-001-03-7
 - total samples: 461
 - train samples: 369
 - test samples : 92

Drug BRD-A06627858-236-03-0
 - total samples: 703
 - train samples: 563
 - test samples : 140

Drug BRD-A08545410-003-07-8
 - total samples: 469
 - train samples: 376
 - test samples : 93

Drug BRD-A22707317-001-10-8
 - total samples: 462
 - train samples: 370
 - test samples : 92

Drug BRD-A49035384-003-28-9
 - total samples: 460
 - train samples: 368
 - test samples : 92

✅ Cell 03 passed: train/test strategy defined and validated.


In [ ]:
# Select evaluation drugs (first 5 from the MVP list, fixed order)
evaluation_drugs = (
    selected_drugs_df
    .sort_values("broad_id")
    .head(5)["broad_id"]
    .tolist()
)

print("Evaluation drugs (n=5):")
for d in evaluation_drugs:
    print(" -", d)

In [ ]:
# Helper: build train/test masks per drug
def build_train_test_masks(df, test_size=0.2, random_state=42):
    """
    Returns boolean masks (train_mask, test_mask) for a given
    PRISM slice corresponding to a single drug.
    """
    rng = np.random.RandomState(random_state)
    idx = np.array(df.index)
    rng.shuffle(idx)

    n_test = int(len(idx) * test_size)
    test_idx = idx[:n_test]
    train_idx = idx[n_test:]

    train_mask = df.index.isin(train_idx)
    test_mask = df.index.isin(test_idx)

    return train_mask, test_mask

In [ ]:
# Sanity check per drug (no modeling yet)
for broad_id in evaluation_drugs:
    sub = prism_mvp[prism_mvp["broad_id"] == broad_id]

    train_mask, test_mask = build_train_test_masks(
        sub,
        test_size=TEST_SIZE,
        random_state=RANDOM_STATE,
    )

    print(f"\nDrug {broad_id}")
    print(" - total samples:", sub.shape[0])
    print(" - train samples:", train_mask.sum())
    print(" - test samples :", test_mask.sum())

    if train_mask.sum() == 0 or test_mask.sum() == 0:
        raise ValueError(f"Invalid split for drug {broad_id}")

print("\n✅ Cell 03 passed: train/test strategy defined and validated.")

## Feature Assembly and Target Definition

In this section, molecular features (gene expression) and drug response targets
are assembled into model-ready matrices.

The feature space is kept identical to previous notebooks to ensure consistency,
and no feature selection or dimensionality reduction is applied at this stage.

## Assemble features (X) and target (y) per drug

In [ ]:
expr_tmp = expr_df.copy()

Expression raw shape: (751, 19220)
Numeric-only shape  : (751, 19216)

Expression matrix after collapsing:
 - shape: (727, 19216)
 - unique cell lines: 727

Drug BRD-A06352508-001-03-7
 - X shape: (461, 19216)
 - y shape: (461,)
 - y AUC range: (np.float64(0.528), np.float64(1.628))

Drug BRD-A06627858-236-03-0
 - X shape: (467, 19216)
 - y shape: (467,)
 - y AUC range: (np.float64(0.106), np.float64(1.425))

Drug BRD-A08545410-003-07-8
 - X shape: (469, 19216)
 - y shape: (469,)
 - y AUC range: (np.float64(0.647), np.float64(2.191))

Drug BRD-A22707317-001-10-8
 - X shape: (462, 19216)
 - y shape: (462,)
 - y AUC range: (np.float64(0.699), np.float64(1.626))

Drug BRD-A49035384-003-28-9
 - X shape: (460, 19216)
 - y shape: (460,)
 - y AUC range: (np.float64(0.519), np.float64(2.078))

✅ Cell 04 passed: X/y assembled (numeric-only expression collapsed per cell line).


In [ ]:
# Normalize depmap_id to index
if "depmap_id" in expr_tmp.columns:
    expr_tmp = expr_tmp.set_index("depmap_id")
else:
    if expr_tmp.index.name is None:
        expr_tmp.index.name = "depmap_id"

In [ ]:
# Keep only numeric feature columns (genes)
expr_numeric = expr_tmp.select_dtypes(include="number")

if expr_numeric.shape[1] == 0:
    raise ValueError(
        "No numeric columns found in expression dataframe after select_dtypes(include='number'). "
        "Expression features should be numeric."
    )

print("Expression raw shape:", expr_tmp.shape)
print("Numeric-only shape  :", expr_numeric.shape)

In [ ]:
# Collapse to one vector per cell line (mean across duplicates)
expr_df_collapsed = expr_numeric.groupby(expr_numeric.index, sort=False).mean()

print("\nExpression matrix after collapsing:")
print(" - shape:", expr_df_collapsed.shape)
print(" - unique cell lines:", expr_df_collapsed.index.nunique())

In [ ]:
# Assemble X/y per drug
datasets: Dict[str, Dict[str, Tuple[pd.DataFrame, pd.Series]]] = {}

for broad_id in evaluation_drugs:
    # Subset PRISM for this drug
    prism_sub = prism_mvp[prism_mvp["broad_id"] == broad_id].copy()

    # --- Collapse PRISM to one AUC per cell line (MVP decision)
    prism_sub = (
        prism_sub
        .groupby("depmap_id", as_index=False)["auc"]
        .mean()
    )

    common_cells = prism_sub["depmap_id"].unique()

    # Align features
    X = expr_df_collapsed.loc[common_cells]

    # Target aligned to X
    y = (
        prism_sub
        .set_index("depmap_id")
        .loc[X.index, "auc"]
    )

    # Sanity checks (same as before)
    if X.shape[0] != y.shape[0]:
        raise ValueError(f"Sample mismatch for drug {broad_id}")

    if X.isna().any().any():
        raise ValueError(f"NaNs detected in X for drug {broad_id}")

    if y.isna().any():
        raise ValueError(f"NaNs detected in y for drug {broad_id}")

    datasets[broad_id] = {"X": X, "y": y}

    print(f"\nDrug {broad_id}")
    print(" - X shape:", X.shape)
    print(" - y shape:", y.shape)
    print(" - y AUC range:", (round(y.min(), 3), round(y.max(), 3)))


print("\n✅ Cell 04 passed: X/y assembled (numeric-only expression collapsed per cell line).")

## Baseline Model Training

This section trains simple baseline models to establish whether the current
feature space contains predictive signal for drug response.

Models are trained independently per drug using the fixed train/test split
defined above, with minimal preprocessing and no hyperparameter tuning.

## Baseline training + evaluation metrics

In [ ]:
results = []


=== Drug: BRD-A06352508-001-03-7 ===
Train: (369, 19216) Test: (92, 19216)
ridge         | R2: -0.204 | RMSE:  0.179 | Spearman:  0.131
elasticnet    | R2: -0.003 | RMSE:  0.163 | Spearman:  0.002
random_forest | R2:  0.019 | RMSE:  0.161 | Spearman:  0.112

=== Drug: BRD-A06627858-236-03-0 ===
Train: (374, 19216) Test: (93, 19216)
ridge         | R2: -0.542 | RMSE:  0.175 | Spearman:  0.104
elasticnet    | R2: -0.076 | RMSE:  0.146 | Spearman: -0.001
random_forest | R2: -0.203 | RMSE:  0.155 | Spearman:  0.162

=== Drug: BRD-A08545410-003-07-8 ===
Train: (376, 19216) Test: (93, 19216)
ridge         | R2: -0.118 | RMSE:  0.154 | Spearman:  0.169
elasticnet    | R2:  0.007 | RMSE:  0.145 | Spearman:  0.042
random_forest | R2: -0.059 | RMSE:  0.150 | Spearman:  0.075

=== Drug: BRD-A22707317-001-10-8 ===
Train: (370, 19216) Test: (92, 19216)
ridge         | R2: -0.307 | RMSE:  0.197 | Spearman:  0.226
elasticnet    | R2: -0.024 | RMSE:  0.174 | Spearman:  0.022
random_forest | R2:  0.06

,broad_id,model,n_train,n_test,r2_test,rmse_test,spearman_test
1,BRD-A06352508-001-03-7,elasticnet,369,92,-0.003022,0.163177,0.002004
2,BRD-A06352508-001-03-7,random_forest,369,92,0.018622,0.161407,0.111616
0,BRD-A06352508-001-03-7,ridge,369,92,-0.204166,0.178792,0.131390
4,BRD-A06627858-236-03-0,elasticnet,374,93,-0.076410,0.146244,-0.000746
5,BRD-A06627858-236-03-0,random_forest,374,93,-0.202859,0.154596,0.162409
3,BRD-A06627858-236-03-0,ridge,374,93,-0.542445,0.175063,0.104473
7,BRD-A08545410-003-07-8,elasticnet,376,93,0.006707,0.145028,0.042165
8,BRD-A08545410-003-07-8,random_forest,376,93,-0.059469,0.149782,0.075423
6,BRD-A08545410-003-07-8,ridge,376,93,-0.118269,0.153882,0.168765
10,BRD-A22707317-001-10-8,elasticnet,370,92,-0.023864,0.174159,0.022178



✅ Cell 05 passed: baseline models trained and evaluated.


In [ ]:
# Define baseline models (minimal, no tuning)
models = {
    "ridge": Ridge(random_state=RANDOM_STATE),
    "elasticnet": ElasticNet(random_state=RANDOM_STATE, max_iter=5000),
    "random_forest": RandomForestRegressor(
        n_estimators=300,
        random_state=RANDOM_STATE,
        n_jobs=-1
    ),
}

In [ ]:
for broad_id in evaluation_drugs:
    X = datasets[broad_id]["X"]
    y = datasets[broad_id]["y"]

    # Build split masks on an aligned dataframe (index-based)
    split_df = pd.DataFrame({"auc": y}, index=X.index)
    train_mask, test_mask = build_train_test_masks(
        split_df,
        test_size=TEST_SIZE,
        random_state=RANDOM_STATE,
    )

    X_train, X_test = X.loc[train_mask], X.loc[test_mask]
    y_train, y_test = y.loc[train_mask], y.loc[test_mask]

    print(f"\n=== Drug: {broad_id} ===")
    print("Train:", X_train.shape, "Test:", X_test.shape)

    for model_name, model in models.items():
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        r2 = r2_score(y_test, y_pred)
        rmse = mean_squared_error(y_test, y_pred) ** 0.5
        rho = spearmanr(y_test, y_pred).correlation

        results.append({
            "broad_id": broad_id,
            "model": model_name,
            "n_train": X_train.shape[0],
            "n_test": X_test.shape[0],
            "r2_test": r2,
            "rmse_test": rmse,
            "spearman_test": rho,
        })

        print(f"{model_name:13s} | R2: {r2: .3f} | RMSE: {rmse: .3f} | Spearman: {rho: .3f}")

results_df = pd.DataFrame(results).sort_values(["broad_id", "model"])
display(results_df)

print("\n✅ Cell 05 passed: baseline models trained and evaluated.")